# **Section 1: Environment Setup & Dependencies**
In this section, we configure the runtime environment, mount persistent storage (Google Drive), and authenticate with Hugging Face to access the required datasets.

### **1. Project Setup & Directory Initialization**
We initialize the project structure by creating the necessary subdirectories for raw data, processed features, metadata, and model checkpoints.

In [ ]:
!mkdir -p /content/drive/MyDrive/hindi_dfake/{raw,processed,metadata,scripts,checkpoints}

### **2. Environment Configuration**
We mount Google Drive to ensure persistent storage and dynamically define the root paths (`ROOT_DIR`, `OUT_DIR`, etc.) to avoid hardcoding errors across different sessions.

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

import os, glob
MOUNT_PATH = "/content/drive" if os.path.isdir("/content/drive/MyDrive") else "/content/gdrive"
print("Drive is already mounted here →", MOUNT_PATH)
ROOT_DIR = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR  = f"{ROOT_DIR}/raw/real_clean/ivr"
CSV_PATH = f"{ROOT_DIR}/metadata/master_real.csv"

Mounted at /content/drive
Drive is already mounted here → /content/drive


### **3. Hugging Face Authentication**
Authentication is required to download the **IndicVoices-R** and **Common Voice** datasets directly from the Hugging Face Hub.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# **Section 2: Authentic Dataset Construction**
We aggregate and standardize authentic Hindi speech from diverse sources—including **CommonVoice**, **IndicTTS** **IndicVoices**, and **FLEURS**—to establish a robust "real" baseline for our deepfake detection model.

### **1. Library Dependencies**
We install specific versions of `pandas`, `requests`, and the Hugging Face `datasets` library. Pinning these versions ensures compatibility with the data download scripts and prevents dependency conflicts.

In [ ]:
!pip install -Uq pandas==2.2.2 requests==2.32.3

!pip -q install --force-reinstall --no-deps "datasets==2.19.2"

!pip -q install pyarrow-hotfix "fsspec[http]==2024.3.1" --no-deps

### **2. IndicVoices-R**
We stream 30 hours of Hindi audio from **IndicVoices-R**, filtering for valid transcripts. All files are resampled to **16kHz** and saved as WAVs with a metadata CSV.

In [ ]:
import os, csv, tqdm, hashlib, glob
import soundfile as sf
from datasets import load_dataset, Audio

HOURS_TO_SAVE = 30
OUT_DIR       = f"{ROOT_DIR}/raw/real_clean/ivr"
CSV_PATH      = f"{ROOT_DIR}/metadata/master_real.csv"
TEXT_KEY      = "text"

# create the folders once
os.makedirs(OUT_DIR,      exist_ok=True)
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)

# --- SMART CHECK: Verify BOTH Audio and Metadata exist ---
existing_files = glob.glob(f"{OUT_DIR}/*.wav")
csv_exists = os.path.exists(CSV_PATH)

if len(existing_files) > 100 and csv_exists:
    # Only skip if we have audio AND the csv
    print(f"📂 Found {len(existing_files)} existing files and metadata CSV.")
    print("🔄 Verifying duration...")
    total_duration = 0
    for f in tqdm.tqdm(existing_files, unit="file"):
        try: total_duration += sf.info(f).duration
        except: pass

    secs_done = total_duration
    print(f"✅ Data and CSV already present ({secs_done/3600:.1f}h). Skipping download.")

else:
    if len(existing_files) > 100 and not csv_exists:
        print("⚠️ Warning: WAV files found but CSV is missing. Re-running to regenerate metadata...")

    # --- YOUR ORIGINAL CODE STARTS HERE ---
    ds_iter = (
        load_dataset("ai4bharat/indicvoices_r", "Hindi",
                    split="train", streaming=True)
        .cast_column("audio", Audio(sampling_rate=16_000))
        .filter(lambda x: len(x[TEXT_KEY].strip()) > 0)
    )

    secs_cap  = HOURS_TO_SAVE * 3600
    secs_done = 0

    with open(CSV_PATH, "w", newline='') as csv_f, \
        tqdm.tqdm(total=secs_cap, unit="s") as bar:

        writer = csv.DictWriter(csv_f,
                fieldnames=["utt_id", "path", "duration", "speaker_id",
                            "gender", "text", "label"])
        writer.writeheader()

        for ex in ds_iter:
            arr, sr = ex["audio"]["array"], ex["audio"]["sampling_rate"]
            dur     = len(arr) / sr
            utt_id  = hashlib.sha1(ex["audio"]["path"].encode()).hexdigest()[:16]
            out_wav = f"{OUT_DIR}/{utt_id}.wav"

            sf.write(out_wav, arr, sr)
            writer.writerow({
                "utt_id"     : utt_id,
                "path"       : out_wav,
                "duration"   : dur,
                "speaker_id" : ex["speaker_id"],
                "gender"     : ex["gender"],
                "text"       : ex[TEXT_KEY],
                "label"      : "real"
            })

            secs_done += dur
            bar.update(dur)
            if secs_done >= secs_cap:
                break

# Print final status
print(f"\nSaved ≈ {secs_done/3600:.1f} h of audio to Drive:")
print("  WAVs →", OUT_DIR)
print("  CSV  →", CSV_PATH)

📂 Found 11174 existing files and metadata CSV.
🔄 Verifying duration...


100%|██████████| 11174/11174 [01:56<00:00, 95.78file/s] 

✅ Data and CSV already present (30.0h). Skipping download.

Saved ≈ 30.0 h of audio to Drive:
  WAVs → /content/drive/MyDrive/hindi_dfake/raw/real_clean/ivr
  CSV  → /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv


### **3. IndicTTS-Hindi**
We acquire approximately 10.5 hours of high-quality studio speech from the **IndicTTS-Hindi** dataset.


In [ ]:
# ==============================================================
#  IndicTTS-Hindi  →  ~10 h studio-quality real speech
#  • Writes WAVs to  .../raw/real_clean/indictts
#  • Appends rows to .../metadata/master_real.csv
# ==============================================================

# ---------- imports & dirs -------------------------------------
import os, csv, hashlib, tqdm, glob
import soundfile as sf
from datasets import load_dataset, Audio

# ---------- paths (edit ONLY if your mount path differs) -------
MOUNT_PATH   = "/content/drive"              # or "/content/gdrive"
ROOT_DIR     = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR      = f"{ROOT_DIR}/raw/real_clean/indictts"
CSV_PATH     = f"{ROOT_DIR}/metadata/master_real.csv"

# ---------- constants ------------------------------------------
TARGET_HOURS = 10.5           # whole dataset ≈10.3 h (leave as-is)
GENDER_MAP   = {0: "m", 1: "f", 2: "o"}      # fallback "u"
TEXT_KEY     = "text"         # transcript field in IndicTTS

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)

# --- SMART CHECK: Verify Audio exists (Note: We append to CSV, so we check WAV count only) ---
existing_files = glob.glob(f"{OUT_DIR}/*.wav")

if len(existing_files) > 100:
    # SUCCESS PATH: Data exists, just print success message
    print(f"📂 Found {len(existing_files)} existing IndicTTS files.")
    print("🔄 Verifying duration...")
    total_duration = 0
    for f in tqdm.tqdm(existing_files, unit="file"):
        try: total_duration += sf.info(f).duration
        except: pass

    secs_done = total_duration
    print(f"✅ IndicTTS data already present ({secs_done/3600:.1f}h). Skipping download.")

else:
    # --- ORIGINAL CODE STARTS HERE ---

    # ---------- helper: write header only once ---------------------
    need_header = not os.path.isfile(CSV_PATH)

    # ---------- streaming iterator ---------------------------------
    tts_iter = (load_dataset("SPRINGLab/IndicTTS-Hindi",
                            split="train", streaming=True)
                .cast_column("audio", Audio(sampling_rate=16_000)))

    secs_cap  = TARGET_HOURS * 3600
    secs_done = 0

    with open(CSV_PATH, "a", newline='') as csv_f, \
        tqdm.tqdm(total=secs_cap, unit="s", desc="IndicTTS DL") as bar:

        writer = csv.DictWriter(csv_f,
            fieldnames=["utt_id","path","duration","speaker_id",
                        "gender","text","label"])

        if need_header:
            writer.writeheader()

        for ex in tts_iter:
            arr, sr = ex["audio"]["array"], ex["audio"]["sampling_rate"]
            dur     = len(arr) / sr
            utt_id  = hashlib.sha1(ex["audio"]["path"].encode()).hexdigest()[:16]

            # gender mapping (numeric → letter)
            g_val   = ex.get("gender", "u")
            g_letter = GENDER_MAP.get(g_val, "u")

            out_wav = f"{OUT_DIR}/{utt_id}.wav"
            sf.write(out_wav, arr, sr)

            writer.writerow({
                "utt_id"    : utt_id,
                "path"      : out_wav,
                "duration"  : dur,
                "speaker_id": f"{g_letter}_{utt_id[:4]}",
                "gender"    : g_letter,
                "text"      : ex[TEXT_KEY],
                "label"     : "real"
            })

            secs_done += dur
            bar.update(dur)
            if secs_done >= secs_cap:
                break

# Print final status
print(f"\n✓ IndicTTS download finished — {secs_done/3600:.2f} h saved to {OUT_DIR}")
print("  Metadata appended to →", CSV_PATH)

📂 Found 4703 existing IndicTTS files.
🔄 Verifying duration...


100%|██████████| 4703/4703 [01:50<00:00, 42.57file/s] 

✅ IndicTTS data already present (10.5h). Skipping download.

✓ IndicTTS download finished — 10.50 h saved to /content/drive/MyDrive/hindi_dfake/raw/real_clean/indictts
  Metadata appended to → /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv


### **4. Common Voice**
We acquire 15 hours of validated Hindi speech from **Mozilla Common Voice**, filtering for quality, duration, and community votes.

In [ ]:
CV_REPO = "mozilla-foundation/common_voice_17_0"   # ← change to 17_0 if you logged in
CV_LANG = "hi"                                     # Hindi config name
TOKEN_KW = {}                                      # populated only if you logged in
# If you ran `from huggingface_hub import login; login("YOUR_TOKEN")`
# and want v17, uncomment the next line:
# TOKEN_KW = {"token": True, "trust_remote_code": True}

# ─── local paths ────────────────────────────────────────────────────────
MOUNT_PATH  = "/content/drive"                      # or "/content/gdrive"
ROOT_DIR    = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR     = f"{ROOT_DIR}/raw/real_clean/commonvoice"
CSV_PATH    = f"{ROOT_DIR}/metadata/master_real.csv"

# ─── constants ──────────────────────────────────────────────────────────
TARGET_HOURS = 15
TEXT_KEY     = "sentence"
MIN_UVOTES   = 1           # ≥1 community up-vote
DUR_RANGE    = (1.0, 8.0)  # 1–8 s clips
ENERGY_TH    = 1e-4        # discard near-silence
GENDER_MAP   = {"male":"m","female":"f","other":"o"}

# ─── imports & setup ────────────────────────────────────────────────────
import os, csv, tqdm, hashlib, numpy as np, soundfile as sf
from datasets import load_dataset, Audio
os.makedirs(OUT_DIR, exist_ok=True)
need_header = not os.path.isfile(CSV_PATH)

# helper: simple RMS energy
def rms(arr): return np.sqrt(np.mean(arr.astype(np.float32)**2))

# filter for duration, up-votes, energy
def cv_filter(ex):
    dur = ex["audio"]["array"].shape[-1] / 16_000
    if not (DUR_RANGE[0] <= dur <= DUR_RANGE[1]): return False
    if ex.get("up_votes",0) < MIN_UVOTES:         return False
    if rms(ex["audio"]["array"]) < ENERGY_TH:     return False
    return True

# streaming iterator
cv_iter = (
    load_dataset(CV_REPO, CV_LANG,
                 split="validated", streaming=True,
                 **TOKEN_KW)
    .cast_column("audio", Audio(sampling_rate=16_000))
    .filter(cv_filter)
)

# ─── download loop ──────────────────────────────────────────────────────
secs_cap  = TARGET_HOURS * 3600
secs_done = 0

with open(CSV_PATH, "a", newline='') as csv_f, \
     tqdm.tqdm(total=secs_cap, unit="s", desc="CommonVoice DL") as bar:

    writer = csv.DictWriter(csv_f,
        fieldnames=["utt_id","path","duration","speaker_id",
                    "gender","text","label"])
    if need_header: writer.writeheader()

    for ex in cv_iter:
        arr = ex["audio"]["array"]; sr = 16_000
        dur = len(arr) / sr
        utt_id  = hashlib.sha1(ex["path"].encode()).hexdigest()[:16]
        gender  = GENDER_MAP.get(str(ex.get("gender","")).lower(), "u")

        out_wav = f"{OUT_DIR}/{utt_id}.wav"
        sf.write(out_wav, arr, sr)

        writer.writerow({
            "utt_id"   : utt_id,
            "path"     : out_wav,
            "duration" : dur,
            "speaker_id": ex.get("client_id","anon"),
            "gender"   : gender,
            "text"     : ex[TEXT_KEY],
            "label"    : "real"
        })

        secs_done += dur
        bar.update(dur)
        if secs_done >= secs_cap: break

print(f"\n✓ Common Voice download finished — {secs_done/3600:.2f} h saved to {OUT_DIR}")
print("  Metadata appended to →", CSV_PATH)


/usr/local/lib/python3.11/dist-packages/datasets/load.py:1491: FutureWarning: The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


CommonVoice DL:   0%|          | 0/54000 [00:00<?, ?s/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 1it [00:00,  2.49it/s]
Reading metadata...: 10329it [00:00, 18452.01it/s]
CommonVoice DL:  84%|████████▍ | 45447.37199999915/54000 [03:38<00:41, 208.03s/s]


✓ Common Voice download finished — 12.62 h saved to /content/drive/MyDrive/hindi_dfake/raw/real_clean/commonvoice
  Metadata appended to → /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv


### **5. Google FLEURS**
We acquire a supplementary test set from **Google FLEURS (Hindi)**, targeting up to 3000 clips (or maximum available) from all splits to ensure speaker diversity.

In [ ]:
# === Colab one-cell: Bulk real Hindi (FLEURS hi_in) ~3000 clips ===
# Safe: writes ONLY to new folders; does NOT modify your existing CSVs.

# ---------- config ----------
ROOT         = "/content/drive/MyDrive/hindi_dfake"
OUT_DIR      = f"{ROOT}/raw/real_clean/thirdparty/fleurs_hi_in"
CSV_OUT      = f"{ROOT}/metadata/thirdparty_real_test.fleurs.csv"
N_TARGET     = 3000
RANDOM_SEED  = 1337
DUR_RANGE_S  = (1.0, 8.0)        # keep 1–8s
SOURCE_TAG   = "google/fleurs:hi_in"
LICENSE_TAG  = "CC-BY-4.0"

# ---------- setup ----------
import os, csv, random, hashlib, numpy as np, soundfile as sf
from pathlib import Path

# Mount Drive if needed
if not os.path.exists("/content/drive"):
    from google.colab import drive
    drive.mount("/content/drive")

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(Path(CSV_OUT).parent, exist_ok=True)

# clean install datasets (keeps Colab pins)
!pip -q install datasets==2.19.0 > /dev/null

from datasets import load_dataset, Audio

# ---------- load & filter ----------
print("[load] FLEURS hi_in (train+validation+test)…")
ds = load_dataset("google/fleurs", "hi_in", split="train+validation+test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

MIN_S, MAX_S = DUR_RANGE_S
def good_ex(ex):
    a = ex["audio"]["array"]; sr = ex["audio"]["sampling_rate"]
    if a is None: return False
    d = len(a) / float(sr)
    return (MIN_S <= d <= MAX_S)

ds = ds.filter(good_ex)
print(f"[info] after dur filter: {len(ds)} rows")

# ---------- prefer unique speakers first ----------
rng = random.Random(RANDOM_SEED)
idx_all = list(range(len(ds)))
# build buckets by speaker if present
spk_col = "speaker_id" if "speaker_id" in ds.column_names else None
if spk_col:
    from collections import defaultdict
    by_spk = defaultdict(list)
    for i in idx_all:
        by_spk[str(ds[i].get(spk_col, f"spk?"))].append(i)
    speakers = list(by_spk.keys())
    rng.shuffle(speakers)

    chosen_idxs = []
    # 1) take one per speaker (unique speakers pass)
    for s in speakers:
        chosen_idxs.append(by_spk[s][0])
        if len(chosen_idxs) >= N_TARGET:
            break

    # 2) backfill (allow multiple per speaker) if needed
    if len(chosen_idxs) < N_TARGET:
        leftovers = []
        for s in speakers:
            leftovers.extend(by_spk[s][1:])  # remaining clips per spk
        rng.shuffle(leftovers)
        need = N_TARGET - len(chosen_idxs)
        chosen_idxs.extend(leftovers[:need])

else:
    # no speaker ids → just random sample up to N_TARGET
    rng.shuffle(idx_all)
    chosen_idxs = idx_all[:N_TARGET]

print(f"[pick] selected {len(chosen_idxs)} clips (target={N_TARGET})")

# ---------- save WAVs + CSV (standalone) ----------
def norm_text(ex):
    # FLEURS has "transcription" and "raw_transcription"
    return ex.get("transcription") or ex.get("raw_transcription") or ""

def to_wav_and_row(ex, idx):
    arr = ex["audio"]["array"]; sr = ex["audio"]["sampling_rate"]  # 16k after cast
    if arr.ndim > 1:
        arr = arr.mean(axis=1)
    arr = np.clip(arr.astype(np.float32), -1.0, 1.0)
    secs = round(len(arr)/float(sr), 3)

    spk = str(ex.get(spk_col, f"spk_{idx}"))
    # stable utt_id from original path if available, else from content hash
    src_path = str(ex.get("path", f"fleurs_hi_in_{idx}"))
    uid_base = hashlib.sha1(src_path.encode("utf-8")).hexdigest()[:16]
    utt_id = f"fleurs_hi_{uid_base}"

    out_wav = Path(OUT_DIR) / f"{utt_id}.wav"
    sf.write(str(out_wav), arr, sr, subtype="PCM_16")

    row = {
        "utt_id": utt_id,
        "path": str(out_wav).replace("\\","/"),
        "duration": secs,
        "speaker_id": spk,
        "gender": "u",
        "text": norm_text(ex),
        "label": "real",
        "source": SOURCE_TAG,
        "license": LICENSE_TAG
    }
    return row

rows = []
kept = 0
seen_paths = set()
for i in chosen_idxs:
    ex = ds[i]
    r = to_wav_and_row(ex, i)
    # avoid accidental duplicate paths
    if r["path"] in seen_paths:
        continue
    seen_paths.add(r["path"])
    rows.append(r); kept += 1

print(f"[save] writing CSV → {CSV_OUT}")
with open(CSV_OUT, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=[
        "utt_id","path","duration","speaker_id","gender","text","label","source","license"
    ])
    w.writeheader()
    w.writerows(rows)

# ---------- summary ----------
total_sec = sum(r["duration"] for r in rows)
print("\n[summary]")
print(f"  WAVs saved : {kept}")
print(f"  Hours      : {total_sec/3600:.2f}")
print(f"  Out folder : {OUT_DIR}")
print(f"  CSV        : {CSV_OUT}")

# show a few examples
from itertools import islice
print("\n[examples]")
for r in islice(rows, 5):
    print(" •", Path(r["path"]).name, "|", r["speaker_id"], "|", r["duration"], "s")

[load] FLEURS hi_in (train+validation+test)…


/usr/local/lib/python3.12/dist-packages/datasets/load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Filter:   0%|          | 0/2777 [00:00<?, ? examples/s]

[info] after dur filter: 604 rows
[pick] selected 604 clips (target=3000)
[save] writing CSV → /content/drive/MyDrive/hindi_dfake/metadata/thirdparty_real_test.fleurs.csv

[summary]
  WAVs saved : 604
  Hours      : 1.10
  Out folder : /content/drive/MyDrive/hindi_dfake/raw/real_clean/thirdparty/fleurs_hi_in
  CSV        : /content/drive/MyDrive/hindi_dfake/metadata/thirdparty_real_test.fleurs.csv

[examples]
 • fleurs_hi_8b63ce7acd912273.wav | spk_354 | 7.56 s
 • fleurs_hi_6c47be90259fefd1.wav | spk_36 | 7.68 s
 • fleurs_hi_cb0357a131aedb4f.wav | spk_275 | 6.54 s
 • fleurs_hi_9f5518aed01136d1.wav | spk_284 | 6.18 s
 • fleurs_hi_f973483f16007db9.wav | spk_255 | 5.76 s


### **6. FLEURS Audit & Test Set Generation**
We audit the downloaded FLEURS data to verify file integrity and generate the final `test_real.csv` manifest used for model evaluation.

In [ ]:
# === FLEURS audit + build a dedicated test-real CSV (FLEURS-only for now) ===

import os, pandas as pd, soundfile as sf
from pathlib import Path

ROOT      = "/content/drive/MyDrive/hindi_dfake"
FLEURS_CSV= f"{ROOT}/metadata/thirdparty_real_test.fleurs.csv"   # from your earlier one-cell
TEST_REAL = f"{ROOT}/metadata/test_real.csv"                      # dedicated CSV we’ll build/refresh
REPORT_MISS = True                                                # log missing files if any

assert os.path.exists(os.path.dirname(TEST_REAL)), "metadata folder not found"

if not os.path.isfile(FLEURS_CSV):
    raise FileNotFoundError(f"FLEURS CSV not found at: {FLEURS_CSV}")

fleurs = pd.read_csv(FLEURS_CSV)

# Optional: verify files exist and durations are plausible
exists_mask = fleurs["path"].apply(lambda p: os.path.isfile(p))
missing = fleurs[~exists_mask]
present = fleurs[exists_mask].copy()

# If duration column exists, keep it; else compute quickly (reading headers only)
def probe_dur(p):
    try:
        info = sf.info(p)
        return float(info.duration)
    except Exception:
        return None

if "duration" not in present.columns or present["duration"].isnull().any():
    present["duration"] = present["path"].apply(probe_dur)

# Summary
n_total   = len(fleurs)
n_present = len(present)
n_missing = len(missing)
hrs_total = (present["duration"].fillna(0).sum() / 3600.0)

print("🔎 FLEURS audit")
print(f"  rows in CSV        : {n_total}")
print(f"  files present      : {n_present}")
print(f"  files missing      : {n_missing}")
print(f"  unique speakers    : {present['speaker_id'].astype(str).nunique()}")
print(f"  hours (present)    : {hrs_total:.2f} h")

if REPORT_MISS and n_missing:
    print("\n⚠️ Missing files (showing up to 10):")
    for p in missing["path"].head(10).tolist():
        print("  -", p)

# Build/refresh a dedicated test-real CSV with just FLEURS for now
cols = ["utt_id","path","duration","speaker_id","gender","text","label","source","license"]
for c in cols:
    if c not in present.columns:
        present[c] = "" if c != "duration" else 0.0

present = present[cols].copy()
present["label"] = "real"
present["source"] = present.get("source", "").replace("", "google/fleurs:hi_in")
present["license"] = present.get("license", "").replace("", "CC-BY-4.0")

present.to_csv(TEST_REAL, index=False)
print(f"\n✅ Wrote dedicated test-real CSV (FLEURS only) → {TEST_REAL}")

🔎 FLEURS audit
  rows in CSV        : 604
  files present      : 604
  files missing      : 0
  unique speakers    : 604
  hours (present)    : 1.10 h

✅ Wrote dedicated test-real CSV (FLEURS only) → /content/drive/MyDrive/hindi_dfake/metadata/test_real.csv


### **7. Train/Test Splitting (Real Speech)**
We construct a final test set of ~3,000 samples by combining the unseen **FLEURS** data with a **speaker-disjoint** subset of Common Voice and IndicVoices-R. This ensures that the model is never tested on speakers it saw during training.

In [ ]:
# === Build ~3,000 REAL test set ===
# - Keep existing FLEURS (~600) as-is (needs preprocessing/features later)
# - Move the rest (~2,400) by WHOLE speakers from CV + IVR
# - Ascending by speaker size, round-robin between sources
# - Non-destructive: write sidecar CSVs

import os
import pandas as pd
from collections import deque
from pathlib import Path

ROOT = "/content/drive/MyDrive/hindi_dfake"
MASTER_REAL = f"{ROOT}/metadata/master_real.csv"
FLEURS_CSV  = f"{ROOT}/metadata/thirdparty_real_test.fleurs.csv"   # from your FLEURS cell
OUT_TEST_FROM_TRAIN = f"{ROOT}/metadata/test_real.from_train.roundrobin_least_damage.csv"
OUT_TEST_UNION      = f"{ROOT}/metadata/test_real.union.csv"
OUT_TRAIN_AFTER     = f"{ROOT}/metadata/train_real.after_handoff.roundrobin_least_damage.csv"

TARGET_TEST_TOTAL = 3000
ALLOW_OVERSHOOT   = 0.05   # allow up to +5% because we move whole speakers
SMALL_SPK_MAX_N   = 40     # cap for "small" speakers to avoid mega-speakers
MIN_SPK_N         = 2      # ignore singletons if you prefer (set to 1 to allow)

def detect_source(p):
    p = str(p).lower()
    if "indictts" in p: return "IndicTTS"
    if "ivr" in p or "indicvoices" in p: return "IndicVoices_R"
    if "commonvoice" in p: return "CommonVoice"
    if "fleurs" in p: return "FLEURS"
    return "Other"

# 1) Load master + fleurs (real rows only)
df = pd.read_csv(MASTER_REAL)
df["label"] = df["label"].astype(str).str.lower()
df["source"] = df["path"].apply(detect_source)
df["speaker_id"] = df["speaker_id"].astype(str)
df_real = df[df["label"].eq("real")].copy()

fleurs = pd.DataFrame()
if os.path.isfile(FLEURS_CSV):
    fleurs = pd.read_csv(FLEURS_CSV)
    fleurs["speaker_id"] = fleurs["speaker_id"].astype(str)
    fleurs["source"] = fleurs.get("source", "FLEURS")
    fleurs = fleurs.copy()
else:
    print("⚠️ FLEURS CSV not found; proceeding with 0 FLEURS rows.")

n_fleurs = len(fleurs)
need_from_train = max(0, TARGET_TEST_TOTAL - n_fleurs)
allow_max_total = int(TARGET_TEST_TOTAL * (1 + ALLOW_OVERSHOOT))

print(f"FLEURS present: {n_fleurs} rows")
print(f"Target total test: {TARGET_TEST_TOTAL} → need from CV+IVR: {need_from_train} rows (max allowed total={allow_max_total})")

# 2) Candidate pool = CV + IVR rows from your master (already preprocessed/features exist)
pool = df_real[df_real["source"].isin(["CommonVoice","IndicVoices_R"])].copy()

# per-speaker counts and source
g = (pool.groupby(["speaker_id","source"])
          .agg(n=("utt_id","count"))
          .reset_index())

# filter to "small" speakers first to minimize disruption
g_small = g[(g["n"] >= MIN_SPK_N) & (g["n"] <= SMALL_SPK_MAX_N)].copy()

# build round-robin queues, smallest-first inside each source
queues = {
    "CommonVoice": deque(g_small[g_small["source"].eq("CommonVoice")].sort_values("n")["speaker_id"].tolist()),
    "IndicVoices_R": deque(g_small[g_small["source"].eq("IndicVoices_R")].sort_values("n")["speaker_id"].tolist()),
}

# selection loop: alternate sources to keep mix; stop when need_from_train met
order = ["CommonVoice", "IndicVoices_R"]
chosen_spk = []
chosen_rows = 0
need = need_from_train

# quick map for n per speaker
n_per_spk = { (row["speaker_id"]): int(row["n"]) for _, row in g_small.iterrows() }

while need > 0 and (queues["CommonVoice"] or queues["IndicVoices_R"]):
    for src in order:
        if need <= 0: break
        if queues[src]:
            spk = queues[src].popleft()
            chosen_spk.append(spk)
            chosen_rows += n_per_spk[spk]
            need -= n_per_spk[spk]

# If still under target (e.g., not enough small speakers), backfill with larger speakers (still round-robin)
if chosen_rows < need_from_train:
    remainder = g[(~g["speaker_id"].isin(chosen_spk)) & (g["n"] > SMALL_SPK_MAX_N)].copy()
    queues_big = {
        "CommonVoice": deque(remainder[remainder["source"].eq("CommonVoice")].sort_values("n")["speaker_id"].tolist()),
        "IndicVoices_R": deque(remainder[remainder["source"].eq("IndicVoices_R")].sort_values("n")["speaker_id"].tolist()),
    }
    need2 = need_from_train - chosen_rows
    while need2 > 0 and (queues_big["CommonVoice"] or queues_big["IndicVoices_R"]):
        for src in order:
            if need2 <= 0: break
            if queues_big[src]:
                spk = queues_big[src].popleft()
                chosen_spk.append(spk)
                chosen_rows += int(g.loc[g["speaker_id"].eq(spk), "n"].iloc[0])
                need2 -= int(g.loc[g["speaker_id"].eq(spk), "n"].iloc[0])

print(f"\nSelected speakers: {len(chosen_spk)}")
print(f"Selected rows from CV+IVR: {chosen_rows}  (target from CV+IVR={need_from_train})")

# guard overshoot of TOTAL (FLEURS + moved)
total_after = n_fleurs + chosen_rows
if total_after > allow_max_total:
    print(f"⚠️ Overshoot: total test would be {total_after} (> {allow_max_total}). "
          f"Consider lowering SMALL_SPK_MAX_N or TARGET_TEST_TOTAL.")
else:
    print(f"Projected total test (FLEURS + moved): {total_after} rows")

# 3) Build the splits
move_mask = pool["speaker_id"].isin(chosen_spk)
df_test_from_train = pool[move_mask].copy()
df_train_after     = pd.concat([df_real[~df_real["source"].isin(["CommonVoice","IndicVoices_R"])],  # keep IndicTTS, Other
                               pool[~move_mask]], ignore_index=True)

# union test (FLEURS + moved)
cols = ["utt_id","path","duration","speaker_id","gender","text","label","source","license"]
def ensure_cols(x):
    for c in cols:
        if c not in x.columns:
            x[c] = "" if c != "duration" else 0.0
    return x[cols].copy()

df_test_from_train = ensure_cols(df_test_from_train)
df_test_from_train["label"] = "real"

fleurs_aligned = pd.DataFrame()
if len(fleurs):
    fleurs_aligned = ensure_cols(fleurs.copy())
    fleurs_aligned["label"] = "real"

df_test_union = pd.concat([fleurs_aligned, df_test_from_train], ignore_index=True)

# 4) Save sidecars
Path(OUT_TEST_FROM_TRAIN).parent.mkdir(parents=True, exist_ok=True)
df_test_from_train.to_csv(OUT_TEST_FROM_TRAIN, index=False)
df_test_union.to_csv(OUT_TEST_UNION, index=False)
df_train_after.to_csv(OUT_TRAIN_AFTER, index=False)

# 5) Report + safety
def brief(name, d):
    return {k:int(v) for k,v in d.value_counts().to_dict().items()}

print("\n=== SUMMARY ===")
print(f"FLEURS test (already in CSV): {n_fleurs}")
print(f"Moved from train → test: {len(df_test_from_train)}")
print(f"Total test (union): {len(df_test_union)}")
print(f"Train after handoff: {len(df_train_after)}")
print("\nBy source in moved-to-test:", brief("src", df_test_from_train["source"]))
print("By source in train-after  :", brief("src", df_train_after["source"]))

# Speaker-disjointness check (train-after vs test-union)
assert set(df_test_union["speaker_id"]).isdisjoint(set(df_train_after["speaker_id"])), "Not speaker-disjoint!"
print("\n✅ Speaker-disjointness verified.")

FLEURS present: 604 rows
Target total test: 3000 → need from CV+IVR: 2396 rows (max allowed total=3150)

Selected speakers: 266
Selected rows from CV+IVR: 2418  (target from CV+IVR=2396)
Projected total test (FLEURS + moved): 3022 rows

=== SUMMARY ===
FLEURS test (already in CSV): 604
Moved from train → test: 2418
Total test (union): 3022
Train after handoff: 23547

By source in moved-to-test: {'IndicVoices_R': 1950, 'CommonVoice': 468}
By source in train-after  : {'CommonVoice': 9620, 'IndicVoices_R': 9224, 'IndicTTS': 4703}

✅ Speaker-disjointness verified.
